In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Pará - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [7]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_PA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Pará - Produção de Cimento (t)', axis=1)
data

,Ano,Pará - IDH Renda,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Desemprego,Pará - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Pará - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade,Pará - Consumo de Cimento (t)
0,2008,0.653838,8.343245e+07,4.475828e+06,9.446969,7.069042e+07,8.116333,0.631139,5.483335e+08,2.546644e+09,...,0.680030,35.946490,1008.509202,4848.884846,27.010379,23.052703,80.170200,1.147919,0.762850,1559.491000
1,2009,0.653797,8.418073e+07,4.507563e+06,9.465568,7.086672e+07,8.148790,0.634850,5.302764e+08,2.679945e+09,...,0.681704,36.120974,1008.356065,3110.216649,26.957054,23.034986,80.332906,1.075145,0.764286,1355.544000
2,2010,0.653639,8.675706e+07,4.555784e+06,9.542119,7.180858e+07,8.112071,0.638688,5.046425e+08,2.891930e+09,...,0.683252,36.824311,1008.325688,2155.562456,26.838317,22.916120,80.307326,1.001921,0.765262,1640.682000
3,2011,0.653365,8.890935e+07,4.574157e+06,9.572272,7.235179e+07,8.099477,0.642653,4.854622e+08,3.091988e+09,...,0.684681,37.794614,1008.488595,1742.787440,26.780148,22.842507,80.217883,0.944877,0.765786,1856.176000
4,2012,0.652914,9.072117e+07,4.559887e+06,9.574043,7.265948e+07,9.368278,0.646103,4.867661e+08,3.189506e+09,...,0.685697,37.800824,1008.648003,1488.532092,26.730543,22.851410,80.485786,0.886023,0.765851,2042.910000
5,2013,0.652490,9.239028e+07,4.525097e+06,9.578467,7.295531e+07,9.857304,0.651362,5.225848e+08,3.093872e+09,...,0.687339,36.963360,1008.858495,1334.355904,26.733107,22.837087,80.447713,0.829758,0.765422,2318.608000
6,2014,0.651864,9.370048e+07,4.468711e+06,9.571191,7.308848e+07,10.244256,0.656308,5.676413e+08,2.779340e+09,...,0.688700,36.496952,1009.067756,1232.551825,26.867590,22.874833,80.065651,0.788621,0.764707,2041.730714
7,2015,0.651186,9.493439e+07,4.396932e+06,9.582867,7.332896e+07,10.537304,0.659084,5.179392e+08,2.561731e+09,...,0.689092,36.539513,1009.084655,1168.300890,26.935981,22.950086,80.146850,0.763383,0.763682,1863.549771
8,2016,0.652132,9.599328e+07,4.320024e+06,9.607059,7.362401e+07,10.587975,0.667623,4.524144e+08,2.516859e+09,...,0.694712,38.175099,1009.021941,1146.435512,26.885491,22.912033,80.211878,0.737811,0.771424,1705.680514
9,2017,0.652690,9.681084e+07,4.313294e+06,9.643875,7.397420e+07,10.306831,0.670044,4.213689e+08,2.497710e+09,...,0.695413,41.723496,1009.076536,1076.508642,26.811809,22.889590,80.405874,0.723978,0.770280,1564.940000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Pará - IDH Renda,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Desemprego,Pará - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Pará - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade
0,0.653838,8.343245e+07,4.475828e+06,9.446969,7.069042e+07,8.116333,0.631139,5.483335e+08,2.546644e+09,1.874883e+10,...,11.760983,0.680030,35.946490,1008.509202,4848.884846,27.010379,23.052703,80.170200,1.147919,0.762850
1,0.653797,8.418073e+07,4.507563e+06,9.465568,7.086672e+07,8.148790,0.634850,5.302764e+08,2.679945e+09,2.055015e+10,...,9.472585,0.681704,36.120974,1008.356065,3110.216649,26.957054,23.034986,80.332906,1.075145,0.764286
2,0.653639,8.675706e+07,4.555784e+06,9.542119,7.180858e+07,8.112071,0.638688,5.046425e+08,2.891930e+09,2.342247e+10,...,9.332485,0.683252,36.824311,1008.325688,2155.562456,26.838317,22.916120,80.307326,1.001921,0.765262
3,0.653365,8.890935e+07,4.574157e+06,9.572272,7.235179e+07,8.099477,0.642653,4.854622e+08,3.091988e+09,2.604265e+10,...,10.985689,0.684681,37.794614,1008.488595,1742.787440,26.780148,22.842507,80.217883,0.944877,0.765786
4,0.652914,9.072117e+07,4.559887e+06,9.574043,7.265948e+07,9.368278,0.646103,4.867661e+08,3.189506e+09,2.708753e+10,...,8.159014,0.685697,37.800824,1008.648003,1488.532092,26.730543,22.851410,80.485786,0.886023,0.765851
5,0.652490,9.239028e+07,4.525097e+06,9.578467,7.295531e+07,9.857304,0.651362,5.225848e+08,3.093872e+09,2.523394e+10,...,7.913561,0.687339,36.963360,1008.858495,1334.355904,26.733107,22.837087,80.447713,0.829758,0.765422
6,0.651864,9.370048e+07,4.468711e+06,9.571191,7.308848e+07,10.244256,0.656308,5.676413e+08,2.779340e+09,1.954361e+10,...,10.350442,0.688700,36.496952,1009.067756,1232.551825,26.867590,22.874833,80.065651,0.788621,0.764707
7,0.651186,9.493439e+07,4.396932e+06,9.582867,7.332896e+07,10.537304,0.659084,5.179392e+08,2.561731e+09,1.327596e+10,...,12.454398,0.689092,36.539513,1009.084655,1168.300890,26.935981,22.950086,80.146850,0.763383,0.763682
8,0.652132,9.599328e+07,4.320024e+06,9.607059,7.362401e+07,10.587975,0.667623,4.524144e+08,2.516859e+09,1.011370e+10,...,13.111908,0.694712,38.175099,1009.021941,1146.435512,26.885491,22.912033,80.211878,0.737811,0.771424
9,0.652690,9.681084e+07,4.313294e+06,9.643875,7.397420e+07,10.306831,0.670044,4.213689e+08,2.497710e+09,8.456309e+09,...,9.479812,0.695413,41.723496,1009.076536,1076.508642,26.811809,22.889590,80.405874,0.723978,0.770280


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1355.544000
1     1640.682000
2     1856.176000
3     2042.910000
4     2318.608000
5     2041.730714
6     1863.549771
7     1705.680514
8     1564.940000
9     1375.281000
10    1290.928000
11    1444.547000
12    1529.052000
13    1592.624000
14            NaN
Name: Pará - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Pará - IDH Renda,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Desemprego,Pará - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Pará - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade
0,1.324222,-1.759256,0.299586,-1.981848,-1.870624,-1.353160,-1.576092,1.204752,-0.990234,0.166091,...,0.984445,-1.566920,-0.863255,-0.967463,2.803264,1.936402,1.658039,-0.761793,2.002801,-1.354813
1,1.274511,-1.602011,0.622544,-1.695638,-1.713888,-1.320968,-1.312905,0.820657,-0.546688,0.428911,...,-0.051710,-1.253172,-0.808861,-1.487639,1.218869,1.318238,1.421067,-0.213337,1.493766,-0.784667
2,1.081041,-1.060624,1.113275,-0.517673,-0.876518,-1.357388,-1.040702,0.275392,0.158672,0.847995,...,-0.115145,-0.962925,-0.589599,-1.590821,0.348922,-0.058194,-0.168808,-0.299564,0.981589,-0.397217
3,0.745928,-0.608343,1.300248,-0.053680,-0.393573,-1.369878,-0.759421,-0.132596,0.824346,1.230291,...,0.633403,-0.695012,-0.287112,-1.037461,-0.027227,-0.732514,-1.153417,-0.601063,0.582576,-0.188840
4,0.193042,-0.227609,1.155025,-0.026427,-0.120026,-0.111436,-0.514766,-0.104860,1.148829,1.382743,...,-0.646476,-0.504495,-0.285176,-0.495989,-0.258923,-1.307543,-1.034340,0.301999,0.170908,-0.163203
5,-0.326024,0.123137,0.800981,0.041651,0.142988,0.373598,-0.141809,0.657046,0.830615,1.112297,...,-0.757614,-0.196768,-0.546251,0.219007,-0.399419,-1.277823,-1.225919,0.173660,-0.222645,-0.333625
6,-1.092457,0.398461,0.227158,-0.070314,0.261380,0.757390,0.209035,1.615453,-0.215960,0.282052,...,0.345772,0.058490,-0.691651,0.929824,-0.492190,0.281147,-0.721038,-1.114213,-0.510386,-0.617540
7,-1.922759,0.657755,-0.503318,0.109359,0.475181,1.048046,0.405897,0.558230,-0.940033,-0.632425,...,1.298413,0.131924,-0.678383,0.987226,-0.550740,1.073956,0.285498,-0.840505,-0.686921,-1.024405
8,-0.764233,0.880270,-1.285989,0.481628,0.737493,1.098303,1.011528,-0.835563,-1.089341,-1.093812,...,1.596124,1.185430,-0.168497,0.774201,-0.570665,0.488663,-0.223484,-0.621304,-0.865789,2.050181
9,-0.081515,1.052072,-1.354473,1.048147,1.048832,0.819454,1.183185,-1.495939,-1.153056,-1.335633,...,-0.048438,1.316861,0.937698,0.959647,-0.634387,-0.365485,-0.523664,0.032626,-0.962547,1.596109


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     1355.544000
1     1640.682000
2     1856.176000
3     2042.910000
4     2318.608000
5     2041.730714
6     1863.549771
7     1705.680514
8     1564.940000
9     1375.281000
10    1290.928000
11    1444.547000
Name: Pará - Consumo de Cimento (t), dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Pará - IDH Renda,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Desemprego,Pará - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Pará - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade
12,-0.819242,0.814133,-0.996062,1.239044,0.97042,0.595754,1.119834,1.108405,2.635823,-0.189959,...,-2.260447,0.911811,1.553086,0.035714,-0.656689,-0.219776,2.805206,3.026913,-1.023926,-0.677741


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    1529.052
Name: Pará - Consumo de Cimento (t), dtype: float64

In [16]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [17]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [19]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3497908712, 4291115593, 898035678, 720741760, 3484026664, 2818344525, 2852948252, 73498774, 3679767939, 3300266058, 177097093, 2403548945, 2616385976, 1106491747, 4120377924, 4142364861, 528420489, 2499001357, 2418541396, 4125450527, 1726104939, 1415130099, 4202346683, 439762057, 3044808380, 3735978410, 1151295021, 629246513, 2519214024, 1402798630, 899962529, 4142715338, 2856421120, 1452487184, 3183903839, 1890946678, 1869640651, 382721290, 3103083147, 3875478479, 383073214, 1946671496, 2506534431, 3656186413, 2911009791, 3940565347, 3766106157, 3664995968, 1917861537, 515210989, 1319854278, 3254487851, 1138226969, 4018788152, 1578049241, 1276449927, 3022230279, 1384127914, 2976278716, 3781147835, 2274992369, 3068212678, 3991301787, 1526833216, 1645028573, 2804875276, 4216712997, 2491838419, 1388852200, 1644041349, 2150007820, 943422438, 719130879, 4136566637, 3923771, 3231382265, 2429417689, 3907039170, 4234564336, 3906268357, 3917420005, 3711622101, 801778276, 3406387761, 131209646

2023-09-15 14:55:28.982475: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 14:55:29.014222: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 14:55:29.037623: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 14:55:29.567390: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 14:55:29.594181: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 5.068544387817383


Step: 4 ___________________________________________
val_loss: 0.021421924233436584


Step: 5 ___________________________________________
val_loss: 7238.869140625


Step: 6 ___________________________________________
val_loss: 0.008539095520973206


Step: 7 ___________________________________________
val_loss: 4609.302734375


Step: 8 ___________________________________________
val_loss: 3.25425386428833


Step: 9 ___________________________________________
val_loss: 6.56015157699585


Step: 10 ___________________________________________
val_loss: 197.94964599609375


Step: 11 ___________________________________________
val_loss: 1769.6441650390625


Step: 12 ___________________________________________
val_loss: 0.0037551522254943848
winner_seed: 2616385976


Step: 13 ___________________________________________
val_loss: 6689.3310546875


Step: 14 ___________________________________________
val_loss: 6542.24658203125


Step: 15 _____________________________

In [20]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 1s 524ms/step - loss: 3013813.2500 - val_loss: 1950392.6250
Epoch 2/10000
1/1 [==============================] - 0s 16ms/step - loss: 2960433.2500 - val_loss: 1814968.3750
Epoch 3/10000
1/1 [==============================] - 0s 19ms/step - loss: 2965021.7500 - val_loss: 1954082.1250
Epoch 4/10000
1/1 [==============================] - 0s 16ms/step - loss: 2819544.0000 - val_loss: 2473957.0000
Epoch 5/10000
1/1 [==============================] - 0s 15ms/step - loss: 2698964.2500 - val_loss: 2640445.7500
Epoch 6/10000
1/1 [==============================] - 0s 16ms/step - loss: 2744752.0000 - val_loss: 2405398.7500
Epoch 7/10000
1/1 [==============================] - 0s 16ms/step - loss: 2594659.2500 - val_loss: 2689890.0000
Epoch 8/10000
1/1 [==============================] - 0s 17ms/step - loss: 2756289.2500 - val_loss: 2154777.5000
Epoch 9/10000
1/1 [==============================] - 0s 15ms/step - loss: 2470125.2500 - val_loss: 2644

1/1 [==============================] - 0s 15ms/step - loss: 749911.0625 - val_loss: 328443.2188
Epoch 75/10000
1/1 [==============================] - 0s 15ms/step - loss: 718561.7500 - val_loss: 376345.6250
Epoch 76/10000
1/1 [==============================] - 0s 16ms/step - loss: 723269.8125 - val_loss: 357567.2812
Epoch 77/10000
1/1 [==============================] - 0s 15ms/step - loss: 700722.1875 - val_loss: 341882.6562
Epoch 78/10000
1/1 [==============================] - 0s 15ms/step - loss: 698134.1875 - val_loss: 339498.0625
Epoch 79/10000
1/1 [==============================] - 0s 15ms/step - loss: 685886.9375 - val_loss: 329804.5000
Epoch 80/10000
1/1 [==============================] - 0s 15ms/step - loss: 648009.5625 - val_loss: 298459.7500
Epoch 81/10000
1/1 [==============================] - 0s 15ms/step - loss: 662665.4375 - val_loss: 309320.7812
Epoch 82/10000
1/1 [==============================] - 0s 15ms/step - loss: 650977.9375 - val_loss: 297915.6875
Epoch 83/10000
1

1/1 [==============================] - 0s 14ms/step - loss: 230967.8750 - val_loss: 71206.8906
Epoch 148/10000
1/1 [==============================] - 0s 15ms/step - loss: 227678.9844 - val_loss: 69545.3906
Epoch 149/10000
1/1 [==============================] - 0s 15ms/step - loss: 224469.8750 - val_loss: 68817.0703
Epoch 150/10000
1/1 [==============================] - 0s 15ms/step - loss: 223152.6719 - val_loss: 66222.3516
Epoch 151/10000
1/1 [==============================] - 0s 15ms/step - loss: 217831.3906 - val_loss: 66473.5156
Epoch 152/10000
1/1 [==============================] - 0s 16ms/step - loss: 215115.4219 - val_loss: 67178.4141
Epoch 153/10000
1/1 [==============================] - 0s 15ms/step - loss: 212138.5469 - val_loss: 65882.8984
Epoch 154/10000
1/1 [==============================] - 0s 16ms/step - loss: 209223.4375 - val_loss: 65101.7852
Epoch 155/10000
1/1 [==============================] - 0s 16ms/step - loss: 204725.0469 - val_loss: 60641.7695
Epoch 156/10000
1

Epoch 221/10000
1/1 [==============================] - 0s 15ms/step - loss: 96968.4375 - val_loss: 26795.4766
Epoch 222/10000
1/1 [==============================] - 0s 15ms/step - loss: 99064.9141 - val_loss: 26705.0332
Epoch 223/10000
1/1 [==============================] - 0s 19ms/step - loss: 96506.0391 - val_loss: 25712.4277
Epoch 224/10000
1/1 [==============================] - 0s 20ms/step - loss: 94100.2500 - val_loss: 27711.3984
Epoch 225/10000
1/1 [==============================] - 0s 20ms/step - loss: 98175.9766 - val_loss: 25548.2676
Epoch 226/10000
1/1 [==============================] - 0s 15ms/step - loss: 93156.4609 - val_loss: 23220.7441
Epoch 227/10000
1/1 [==============================] - 0s 15ms/step - loss: 87127.8672 - val_loss: 24285.6387
Epoch 228/10000
1/1 [==============================] - 0s 19ms/step - loss: 92381.6250 - val_loss: 24988.6934
Epoch 229/10000
1/1 [==============================] - 0s 19ms/step - loss: 89818.6875 - val_loss: 25830.4766
Epoch 230/

1/1 [==============================] - 0s 15ms/step - loss: 70843.9766 - val_loss: 9178.5615
Epoch 296/10000
1/1 [==============================] - 0s 16ms/step - loss: 68020.7656 - val_loss: 9605.8906
Epoch 297/10000
1/1 [==============================] - 0s 16ms/step - loss: 66717.5078 - val_loss: 7788.9028
Epoch 298/10000
1/1 [==============================] - 0s 16ms/step - loss: 67689.9531 - val_loss: 9914.0527
Epoch 299/10000
1/1 [==============================] - 0s 15ms/step - loss: 57975.3477 - val_loss: 4959.2231
Epoch 300/10000
1/1 [==============================] - 0s 15ms/step - loss: 67694.1953 - val_loss: 12168.3896
Epoch 301/10000
1/1 [==============================] - 0s 15ms/step - loss: 66124.7891 - val_loss: 7459.8604
Epoch 302/10000
1/1 [==============================] - 0s 15ms/step - loss: 58963.4414 - val_loss: 8688.1875
Epoch 303/10000
1/1 [==============================] - 0s 15ms/step - loss: 66677.1172 - val_loss: 10254.5039
Epoch 304/10000
1/1 [============

...

1/1 [==============================] - 0s 15ms/step - loss: 49933.5508 - val_loss: 1225.8632
Epoch 975/10000
1/1 [==============================] - 0s 16ms/step - loss: 54273.8945 - val_loss: 3906.5247
Epoch 976/10000
1/1 [==============================] - 0s 15ms/step - loss: 53823.4961 - val_loss: 3055.6387
Epoch 977/10000
1/1 [==============================] - 0s 14ms/step - loss: 53776.1562 - val_loss: 3295.5476
Epoch 978/10000
1/1 [==============================] - 0s 14ms/step - loss: 51186.4062 - val_loss: 2261.9939
Epoch 979/10000
1/1 [==============================] - 0s 15ms/step - loss: 83483.2734 - val_loss: 18490.1230
Epoch 980/10000
1/1 [==============================] - 0s 15ms/step - loss: 64368.4258 - val_loss: 7198.0889
Epoch 981/10000
1/1 [==============================] - 0s 15ms/step - loss: 66284.0000 - val_loss: 7698.0273
Epoch 982/10000
1/1 [==============================] - 0s 15ms/step - loss: 66277.1797 - val_loss: 7542.5366
Epoch 983/10000
1/1 [=============

Epoch 1049/10000
1/1 [==============================] - 0s 15ms/step - loss: 67416.9609 - val_loss: 9640.9766
Epoch 1050/10000
1/1 [==============================] - 0s 15ms/step - loss: 67414.3672 - val_loss: 9751.8281
Epoch 1051/10000
1/1 [==============================] - 0s 15ms/step - loss: 66861.8750 - val_loss: 10082.5186
Epoch 1052/10000
1/1 [==============================] - 0s 15ms/step - loss: 67417.2344 - val_loss: 9620.3008
Epoch 1053/10000
1/1 [==============================] - 0s 15ms/step - loss: 67414.1328 - val_loss: 9742.7646
Epoch 1054/10000
1/1 [==============================] - 0s 15ms/step - loss: 67413.7969 - val_loss: 9702.6104
Epoch 1055/10000
1/1 [==============================] - 0s 15ms/step - loss: 67413.3125 - val_loss: 9707.9502
Epoch 1056/10000
1/1 [==============================] - 0s 15ms/step - loss: 67413.5703 - val_loss: 9700.9277
Epoch 1057/10000
1/1 [==============================] - 0s 15ms/step - loss: 67413.4766 - val_loss: 9697.2725
Epoch 105

1/1 [==============================] - 0s 15ms/step - loss: 67409.8047 - val_loss: 9485.0049
Epoch 1124/10000
1/1 [==============================] - 0s 15ms/step - loss: 67409.7891 - val_loss: 9482.8174
Epoch 1125/10000
1/1 [==============================] - 0s 16ms/step - loss: 67409.7578 - val_loss: 9480.6787
Epoch 1126/10000
1/1 [==============================] - 0s 15ms/step - loss: 67409.7344 - val_loss: 9478.5146
Epoch 1127/10000
1/1 [==============================] - 0s 14ms/step - loss: 67409.7109 - val_loss: 9476.4238
Epoch 1128/10000
1/1 [==============================] - 0s 15ms/step - loss: 67409.6797 - val_loss: 9474.3086
Epoch 1129/10000
1/1 [==============================] - 0s 15ms/step - loss: 67409.6641 - val_loss: 9472.2646
Epoch 1130/10000
1/1 [==============================] - 0s 15ms/step - loss: 67409.6328 - val_loss: 9470.1982
Epoch 1131/10000
1/1 [==============================] - 0s 15ms/step - loss: 67409.6172 - val_loss: 9468.1787
Epoch 1132/10000
1/1 [=====

1/1 [==============================] - 0s 15ms/step - loss: 67408.8672 - val_loss: 9378.5938
Epoch 1198/10000
1/1 [==============================] - 0s 16ms/step - loss: 67408.8672 - val_loss: 9378.4043
Epoch 1199/10000
1/1 [==============================] - 0s 15ms/step - loss: 67408.8516 - val_loss: 9377.0811
Epoch 1200/10000
1/1 [==============================] - 0s 15ms/step - loss: 67157.3594 - val_loss: 9761.4492
Epoch 1201/10000
1/1 [==============================] - 0s 19ms/step - loss: 67412.6797 - val_loss: 9271.9131
Epoch 1202/10000
1/1 [==============================] - 0s 19ms/step - loss: 67409.1562 - val_loss: 9406.3242
Epoch 1203/10000
1/1 [==============================] - 0s 16ms/step - loss: 67408.8672 - val_loss: 9367.2012
Epoch 1204/10000
1/1 [==============================] - 0s 16ms/step - loss: 66583.8906 - val_loss: 10160.7725
Epoch 1205/10000
1/1 [==============================] - 0s 15ms/step - loss: 67423.7734 - val_loss: 9153.0146
Epoch 1206/10000
1/1 [====

1/1 [==============================] - 0s 15ms/step - loss: 67410.0000 - val_loss: 9266.3418
Epoch 1272/10000
1/1 [==============================] - 0s 15ms/step - loss: 66446.4688 - val_loss: 9932.9980
Epoch 1273/10000
1/1 [==============================] - 0s 14ms/step - loss: 67417.9922 - val_loss: 9167.4316
Epoch 1274/10000
1/1 [==============================] - 0s 15ms/step - loss: 66775.0625 - val_loss: 9944.1455
Epoch 1275/10000
1/1 [==============================] - 0s 15ms/step - loss: 67418.5000 - val_loss: 9157.7568
Epoch 1276/10000
1/1 [==============================] - 0s 15ms/step - loss: 67409.3984 - val_loss: 9373.9131
Epoch 1277/10000
1/1 [==============================] - 0s 20ms/step - loss: 66758.4375 - val_loss: 9886.9912
Epoch 1278/10000
1/1 [==============================] - 0s 15ms/step - loss: 66743.3594 - val_loss: 9747.1514
Epoch 1279/10000
1/1 [==============================] - 0s 15ms/step - loss: 66360.7969 - val_loss: 9824.6523
Epoch 1280/10000
1/1 [=====

In [21]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 51ms/step
[1529.052](test_target) - [[1444.5481]](prediction) = 84.50390429687491


In [22]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [23]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Pará - IDH Renda,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Desemprego,Pará - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Pará - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade
0,0.930692,-0.953382,-1.132571,-0.920831,-0.879043,-0.573136,-1.217812,1.148205,-1.122043,-1.121418,...,1.412346,-1.240316,-0.924464,1.397253,1.325002,1.044677,0.848136,-1.399021,1.223487,-1.294802
1,0.456803,-0.427906,-0.167173,-0.469129,-0.519886,1.406227,-0.013750,0.140893,-0.184474,-0.185492,...,-0.643250,0.031761,-0.464604,-0.509514,-0.234391,0.303183,0.555982,0.878548,0.002512,0.154840
2,-1.387495,1.381289,1.299744,1.389960,1.398929,-0.833091,1.231562,-1.289098,1.306518,1.306910,...,-0.769096,1.208555,1.389068,-0.887739,-1.090610,-1.347861,-1.404118,0.520474,-1.225999,1.139962


train_target:


,Pará - Consumo de Cimento (t)
0,1355.544
1,1640.682
2,1856.176


test_input:


...

,Pará - IDH Renda,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Desemprego,Pará - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Pará - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade
12,-0.819242,0.814133,-0.996062,1.239044,0.97042,0.595754,1.119834,1.108405,2.635823,-0.189959,...,-2.260447,0.911811,1.553086,0.035714,-0.656689,-0.219776,2.805206,3.026913,-1.023926,-0.677741


test_target:


,Pará - Consumo de Cimento (t)
12,1529.052


1/1 [==============================] - 0s 35ms/step
Error: 84.50390429687491




[1856.1669921875,
 1949.537109375,
 2155.342041015625,
 1995.369140625,
 2063.564453125,
 1705.581298828125,
 1565.1795654296875,
 1535.381103515625,
 1539.30419921875,
 1444.548095703125]

In [24]:
display(targets)
display(predictions)

[2042.91,
 2318.608,
 2041.730714285714,
 1863.549771428571,
 1705.6805142857138,
 1564.94,
 1375.281,
 1290.928,
 1444.547,
 1529.052]

[1856.1669921875,
 1949.537109375,
 2155.342041015625,
 1995.369140625,
 2063.564453125,
 1705.581298828125,
 1565.1795654296875,
 1535.381103515625,
 1539.30419921875,
 1444.548095703125]

In [25]:
mae = mean_absolute_error(predictions, targets)
mae

191.3382604492189

In [26]:
porcentage = mae/np.mean(targets)
porcentage

0.11139065720515827